In [1]:
import pandas as pd
import polars as pl
from transformers import pipeline

c:\Users\antonio.proietti\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# Load file
df = pl.read_excel('data.xlsx')
df = df.to_pandas()
data = df.copy()


In [3]:
data = data[['CID','Nominativo','Mese','Progressivo','Descrizione']]
data.dropna(inplace=True)

In [4]:
data.shape

(4242, 5)

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:

# categories and descriptions
categorie = {
    'Alimentari': ['caffè', 'ristorante', 'snack', 'colazione', 'pranzo', 'cena', 'aperitivo', 'pizzeria', 'bar', 'fast food', 'bevande', 'dolci', 'ristorazione', 'spesa alimentare'],
    'Tassa di soggiorno': ['hotel', 'pernottamento', 'tassa di soggiorno','tassa', 'affitto breve', 'bed and breakfast', 'residence', 'ostello', 'alloggio'],
    'Trasporti': ['biglietto treno', 'taxi', 'volo aereo', 'noleggio auto', 'autobus', 'metropolitana', 'car sharing', 'pedaggi', 'carburante', 'benzina', 'diesel', 'parcheggio','automobile'],
    'Materiale per ufficio': ['cancelleria', 'stampante', 'carta', 'penne', 'scrivania', 'quaderni', 'cartucce', 'computer', 'tastiera', 'mouse', 'monitor', 'forniture ufficio', 'sedie', 'archivio'],
    'Spese di rappresentanza': ['regalo aziendale', 'evento lavoro', 'omaggi per clienti', 'cena di lavoro', 'pranzi con clienti', 'biglietti evento', 'convegno', 'networking','offferto a clienti','offerto a fornitori','offerto a verificatori'],
    'Formazione': ['corso', 'seminario', 'workshop', 'lezione', 'webinar', 'master', 'conferenza', 'certificazione', 'materiale didattico', 'libri di testo'],
    'Telefonia e internet': ['abbonamento telefonico', 'ricarica', 'bolletta internet', 'sim aziendale', 'wifi', 'fibra ottica', 'dati mobili'],
    'Servizi professionali': ['consulenza', 'parcella avvocato', 'parcella commercialista', 'traduzioni', 'servizi legali', 'revisione contabile', 'coaching', 'formazione avanzata'],
    'Marketing e pubblicità': ['campagna pubblicitaria', 'sponsorizzazione', 'social media', 'volantini', 'brochure', 'inserzioni', 'branding', 'eventi promozionali'],
    'Manutenzione': ['riparazione', 'assistenza tecnica', 'manutenzione auto', 'interventi tecnici', 'sostituzione componenti', 'pulizia uffici'],
    'Spese sanitarie': ['visita medica', 'assicurazione sanitaria', 'occhiali', 'medicinali', 'check-up', 'vaccinazioni'],
    'Altro': ['varie', 'spese generiche', 'non specificato', 'extra', 'diverse']
}


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#creating embeddings for each category
categoria_embeddings = {
    categoria: model.encode(frasi) for categoria, frasi in categorie.items()
}

# Function to clsassify expenses using cosine similarity
def classifica_spesa(descrizione):
    if pd.isna(descrizione) or not isinstance(descrizione, str):
        return 'Non classificato'

    descrizione_embedding = model.encode(descrizione)
    similarita_max = 0
    categoria_migliore = 'Altro'

    for categoria, embeddings in categoria_embeddings.items():
        for emb in embeddings:
            similarita = cosine_similarity([descrizione_embedding], [emb])[0][0]
            if similarita > similarita_max:
                similarita_max = similarita
                categoria_migliore = categoria

    return categoria_migliore

In [9]:
data['Descrizione'].head()

2      TASSA DI SOGGIORNO
49        TASSA SOGGIORNO
63        TASSA SOGGIORNO
87     TASSA DI SOGGIORNO
178                 VARIE
Name: Descrizione, dtype: object

In [10]:
classifica_spesa('TASSA DI SOGGIORNO')

'Alimentari'

We need to put everything in lower case

In [23]:
print('\n',classifica_spesa('tassa di soggiorno'),'\n',
    classifica_spesa('hamburger e patatine fritte'),'\n',
    classifica_spesa('riparazione motore'),'\n',)


 Tassa di soggiorno 
 Alimentari 
 Manutenzione 



In [11]:
data = pl.from_pandas(data)

In [12]:
data = data.with_columns(pl.col("Descrizione").map_elements(str.lower))

C:\Users\antonio.proietti\AppData\Local\Temp\ipykernel_16232\3876612162.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  data = data.with_columns(pl.col("Descrizione").map_elements(str.lower))


In [13]:
data = data.with_columns(pl.col("Descrizione").map_elements(classifica_spesa).alias("classificazione"))

C:\Users\antonio.proietti\AppData\Local\Temp\ipykernel_16232\1975405480.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  data = data.with_columns(pl.col("Descrizione").map_elements(classifica_spesa).alias("classificazione"))


In [24]:

# Salva il risultato
data.write_excel('spese_classificate_avanzate.xlsx')
print("Classificazione avanzata completata!")


Classificazione avanzata completata!
